In [211]:
import pandas as pd
from sodapy import Socrata
from uszipcode import SearchEngine # note this does an approximation of zipcode based upon a radius
from datetime import datetime
from datetime import timedelta

import pickle

In [212]:
pd.options.display.max_columns=100

In [213]:
client = Socrata("data.lacity.org", None)

# results = client.get("7fvc-faax", limit=2000000)
results = client.get("7fvc-faax", limit=10000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [214]:
# results_df.head()

In [215]:
# with open('results_df.pickle', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(results_df, f, pickle.HIGHEST_PROTOCOL)

In [216]:
# with open('results_df.pickle', 'rb') as f:
#     # The protocol version used is detected automatically, so we do not
#     # have to specify it.
#     results_df = pickle.load(f)

In [217]:
list(results_df)

[':@computed_region_2dna_qi2s',
 ':@computed_region_k96s_3jcv',
 ':@computed_region_kqwf_mjcx',
 ':@computed_region_qz3q_ghft',
 ':@computed_region_tatf_ua23',
 ':@computed_region_ur2y_g4cx',
 'area_id',
 'area_name',
 'crm_cd',
 'crm_cd_1',
 'crm_cd_2',
 'crm_cd_3',
 'crm_cd_4',
 'crm_cd_desc',
 'cross_street',
 'date_occ',
 'date_rptd',
 'dr_no',
 'location',
 'location_1',
 'mocodes',
 'premis_cd',
 'premis_desc',
 'rpt_dist_no',
 'status',
 'status_desc',
 'time_occ',
 'vict_age',
 'vict_descent',
 'vict_sex',
 'weapon_desc',
 'weapon_used_cd']

In [218]:
rows=len(results_df)
search = SearchEngine(simple_zipcode=True)
lon = []
lat = []
zipcode = []
year_single = '2000'
year = []
for i in range(rows):
    # single values for readability
    lattitude = float(results_df['location_1'][i]['coordinates'][1])
    longitude = float(results_df['location_1'][i]['coordinates'][0])
    year_single = results_df['date_occ'][i][:4]
    # append to arrays
    lon.append(longitude)
    lat.append(lattitude)
    try:
        zipcode.append(search.by_coordinates(lattitude, longitude, radius=5, returns=1)[0].zipcode)
    except:
        zipcode.append('00000')
        print(f'{i},{lattitude},{longitude}')
    year.append(year_single)
results_df['lat'] = lat
results_df['lon'] = lon
results_df['zipcode'] = zipcode
results_df['year'] = year

2513,0.0,0.0


In [219]:
len(results_df)

10000

In [220]:
# remove lines with no corresponding zip code
results_df = results_df[results_df.zipcode != '00000']
len(results_df)

9999

In [221]:
# dates

In [222]:
# construct a basic timedelta and print it
print (timedelta(days=365, hours=0, minutes=0))
# print today's date
print ("today is: " + str(datetime.now()))
# print today's date one year from now
print ("one year from now it will be:" + str(datetime.now() - timedelta(days=365)))


365 days, 0:00:00
today is: 2019-03-22 15:19:34.344063
one year from now it will be:2018-03-22 15:19:34.346062


In [223]:
now = datetime.now()
print(now)
print(timedelta(days=365))
# 1year = datetime.now() - timedelta(days=365)
# 1year
print(type(now))

2019-03-22 15:19:34.355048
365 days, 0:00:00
<class 'datetime.datetime'>


In [242]:
df_0 = results_df

In [230]:
# group by zipcode /crm_cd ; count number of crimes (dr_no) 
aggregation = { 
     'dr_no': 'nunique' 
}
df_1 = results_df.groupby(['zipcode','crm_cd']).agg(aggregation)
df_1

dr_no
zipcode crm_cd       
90001   122         1
        230         1
        310         1
        624         1
        626         3
        627         2
        648         1
        740         1
        761         1
        900         1
90002   230         3
        236         5
        310         2
        354         2
        440         3
        624         5
        625         1
        626         7
        627         1
        648         1
        900         1
        901         1
        930         2
        956         1
90003   110         2
        121         1
        210         2
        230         5
        236         4
        310         4
...               ...
91605   434         1
        440         1
        624         2
        745         1
        760         1
        761         1
        812         1
        860         1
91606   121         1
        210         1
        330         1
        341         1
        354         2
        440         1
        626         1
        940         1
91607   121         1
        310         1
        330         2
        341         1
        420         1
        649         4
        662         1
        903         1
91608   310         1
        354         1
        626         1
        745         1
91744   624         1
91766   210         1

[1615 rows x 1 columns]

In [231]:
# group by year / zipcode ; count number of crimes (dr_no)
aggregation = { 
     'dr_no': 'nunique' 
}
df_2 = results_df.groupby(['year','zipcode']).agg(aggregation)
df_2

dr_no
year zipcode       
2010 90001        3
     90002        6
     90003       13
     90004       69
     90005       11
     90006      150
     90007       11
     90008        4
     90010        8
     90011        4
     90012      523
     90013      508
     90014     1708
     90015      714
     90016       25
     90017      754
     90018       18
     90019       75
     90021      135
     90025        3
     90026      361
     90027        2
     90028        8
     90029      425
     90031       22
     90032        2
     90033        4
     90035        4
     90036       20
     90037       12
...             ...
2019 90012       15
     90013       26
     90014       66
     90015       43
     90016       28
     90017       57
     90018       68
     90021        7
     90023       22
     90026       15
     90029       27
     90031       25
     90032       13
     90033       41
     90037       40
     90042        7
     90043        6
     90057       77
     90062       38
     90063        2
     90071       17
     90079       16
     90248        2
     90710        8
     90717        4
     90731        5
     90732        7
     90744        6
     91030        2
     91402        1

[365 rows x 1 columns]

In [239]:
# group by year / zipcode ; count number of crimes (dr_no)
df_4 = results_df.sort_values(by=['zipcode'])
aggregation = { 
     'dr_no': 'count' 
}
df_3 = df_4.groupby(['zipcode']).agg(aggregation)
df_3 = df_3.sort_values(by=['dr_no'], ascending=False)
df_3 = df_3[:10]
df_3.reset_index(level=0, inplace=True)
df_3

,zipcode,dr_no
0,90014,1798
1,90057,1383
2,90017,827
3,90015,769
4,90012,552
5,90071,544
6,90013,542
7,90029,466
8,90026,383
9,90079,344


In [240]:
df_5 = df_3['zipcode']
df_5

0    90014
1    90057
2    90017
3    90015
4    90012
5    90071
6    90013
7    90029
8    90026
9    90079
Name: zipcode, dtype: object

In [225]:
# rolling year

In [226]:
# 5 years

In [244]:
aggregation = { 
     'dr_no': 'nunique' 
}
df_6={zipcode:pd.DataFrame(df_0[df_0.zipcode == zipcode].groupby(['zipcode','year']).agg(aggregation)) for zipcode in df_5} # works
df_6

{'90014':               dr_no
 zipcode year       
 90014   2010   1708
         2017      2
         2018     22
         2019     66, '90057':               dr_no
 zipcode year       
 90057   2010   1271
         2011      1
         2017      1
         2018     33
         2019     77, '90017':               dr_no
 zipcode year       
 90017   2010    754
         2016      1
         2017      1
         2018     14
         2019     57, '90015':               dr_no
 zipcode year       
 90015   2010    714
         2017      1
         2018     11
         2019     43, '90012':               dr_no
 zipcode year       
 90012   2010    523
         2016      1
         2017      1
         2018     12
         2019     15, '90071':               dr_no
 zipcode year       
 90071   2010    509
         2017      4
         2018     14
         2019     17, '90013':               dr_no
 zipcode year       
 90013   2010    508
         2018      8
         2019     26, '90029':    

In [227]:
# income

In [228]:
# home value

In [229]:
result

SimpleZipcode(zipcode='90079', zipcode_type='Standard', major_city='Los Angeles', post_office_city='Los Angeles, CA', common_city_list=['Los Angeles'], county='Los Angeles County', state='CA', lat=34.053, lng=-118.245, timezone='Pacific', radius_in_miles=0.11363636363636363, area_code_list=['213', '323', '424', '626', '747', '818'], population=0, population_density=None, land_area_in_sqmi=0.0, water_area_in_sqmi=0.0, housing_units=0, occupied_housing_units=0, median_home_value=None, median_household_income=None, bounds_west=-118.256716, bounds_east=-118.254093, bounds_north=34.041599, bounds_south=34.039571)